#### 연습
- data폴더 안에 가전 폴더의 모든 json파일을 하나의 데이터프레임으로 단순 행 결합 
- 데이터에서 결측치를 확인 
    - GeneralPolaruty 컬럼에서 결측치 발견 
- 결측치가 포함된 데이터를 따로 저장 (na_df)
- 결측치를 제거 
- 'RawText', 'GeneralPolarity' 컬럼을 제외한 나머지 컬럼 제외
- 'GeneralPolarity' 컬럼의 이름을 labels 변경 
- RawText는 텍스트 정규화(특수문자 제거, 2칸 이상의 공백 제외, 문자열 앞 뒤 공백 제거)
- labels 데이터에서 -1과 0은 0으로 1은 1로 데이터를 변경 -> 해당 컬럼의 dtype을 int 변경 -> BERTmodel에서 선형 모델로 확률을 예측하기때문에 labels가 위치 값
- train, test의 형태로 데이터를 9:1의 비율로 나눠준다. 
    - labels를 기준으로 계층화 분할
- 데이터프레임을 Dataset의 형태로 변환
- token화 작업은 AutoTokenizer를 이용하여 모델의 이름은 skt/kobert-base-v1 을 이용하여 토큰화 
- 같은 모델을 로드하여 BertModel + Linear 모델 정의 
- Trainer, TrainingArguments를 이용하여 학습 관리 객체 생성
- 학습 -> na_df에서 상위 5개의 RawText을 확인하여 예측

In [167]:
# 파일의 목록을 생성 
import os 
from glob import glob
import pandas as pd 
import torch 
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, BertModel, Trainer, TrainingArguments
import numpy as np

In [168]:
# os 라이브러리를 이용하여 파일의 목록 확인 
file_path = "../data/가전/"
file_list =  os.listdir(file_path)
pd.read_json(file_path + file_list[0])

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspects
0,112038,엄마가 갑자기 전화하시더니 집에서 사용하는 노래방 마이크가 사고 싶다고 하시네요.....,SNS,가전,영상/음향가전,(1+1세트) TJ 태진 블루투스 마이크 / 무선 노래방 마이크,3,323,73,20221110,0.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '우리나라 ..."
1,114572,누가 사용하다가 반품한 것만 같은 제품이 와서 조금 당황스러웠어요ㅠㅠ일단 겉 부분에...,SNS,가전,영상/음향가전,(1등급)삼성 QLED 4K TV 138cm(55형) KQ55QT67AFXKR+삼성...,1,317,70,20221119,-1.0,"[{'Aspect': '품질', 'SentimentText': '겉 부분에 여기저기..."
2,114573,노트북과 TV를 연결해서 모니터 하나에서 다 보려고 구매했어요.여러 제품과 비교하고...,SNS,가전,영상/음향가전,4K HDMI 2.0 양방향 선택기,1,311,70,20221113,0.0,"[{'Aspect': '기능', 'SentimentText': ' 그래도 괜찮은 건..."
3,114574,너무 별로예요 ㅡㅡ… 이 정도 퀄리티인 줄 알았으면 안 샀을 거 같네요… 음질이 거...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,306,84,20221121,-1.0,"[{'Aspect': '품질', 'SentimentText': '너무 별로예요 ㅡㅡ..."
4,114575,소음이 섞여서 나는 편이에요... 사이즈도 작고 휴대하기 간편해서 손이 자주 가긴 ...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,303,73,20221121,-1.0,"[{'Aspect': '소음', 'SentimentText': '소음이 섞여서 나는..."
...,...,...,...,...,...,...,...,...,...,...,...,...
96,114667,이전부터 구매하고 싶어서 계속 눈여겨보다 구매한 락클래식이에요. 포장을 제거하고 제...,SNS,가전,영상/음향가전,엠지텍 락클래식Q9900 (정품),1,290,61,20221120,-1.0,"[{'Aspect': '품질', 'SentimentText': '마감은 좀 문제가 ..."
97,114668,요즘 집에서 작업하면서 핸드폰으로 음악을 들으니 전화를 하거나 핸드폰을 이용할때 자...,SNS,가전,영상/음향가전,오아 아이브릭 휴대용 블루투스 미니 스피커,1,338,78,20221110,-1.0,"[{'Aspect': '디자인', 'SentimentText': '디자인이 좀 그렇..."
98,114669,"처음 들어보는 생소한 브랜드의 tv라 걱정하면서 구입했는데, 역시나 후회 중입니다....",SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,333,79,20221116,-1.0,"[{'Aspect': '소음', 'SentimentText': '별로 소리를 키우지..."
99,114670,최신 기종이라고 해서 기대했는데 구기종보다 못하네요. 소재가 별로여서 예쁜 디자인이...,SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,324,74,20221116,-1.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '최신 기종..."


In [169]:
# glob 라이브러리의 glob를 이용
file_list2 = glob("../data/가전/*.json")
pd.read_json(file_list2[0])

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspects
0,112038,엄마가 갑자기 전화하시더니 집에서 사용하는 노래방 마이크가 사고 싶다고 하시네요.....,SNS,가전,영상/음향가전,(1+1세트) TJ 태진 블루투스 마이크 / 무선 노래방 마이크,3,323,73,20221110,0.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '우리나라 ..."
1,114572,누가 사용하다가 반품한 것만 같은 제품이 와서 조금 당황스러웠어요ㅠㅠ일단 겉 부분에...,SNS,가전,영상/음향가전,(1등급)삼성 QLED 4K TV 138cm(55형) KQ55QT67AFXKR+삼성...,1,317,70,20221119,-1.0,"[{'Aspect': '품질', 'SentimentText': '겉 부분에 여기저기..."
2,114573,노트북과 TV를 연결해서 모니터 하나에서 다 보려고 구매했어요.여러 제품과 비교하고...,SNS,가전,영상/음향가전,4K HDMI 2.0 양방향 선택기,1,311,70,20221113,0.0,"[{'Aspect': '기능', 'SentimentText': ' 그래도 괜찮은 건..."
3,114574,너무 별로예요 ㅡㅡ… 이 정도 퀄리티인 줄 알았으면 안 샀을 거 같네요… 음질이 거...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,306,84,20221121,-1.0,"[{'Aspect': '품질', 'SentimentText': '너무 별로예요 ㅡㅡ..."
4,114575,소음이 섞여서 나는 편이에요... 사이즈도 작고 휴대하기 간편해서 손이 자주 가긴 ...,SNS,가전,영상/음향가전,ASMR 방송용 녹음용 유튜버 핀마이크 삼성/LG스마트폰 C타입 스마트폰 M31-C,1,303,73,20221121,-1.0,"[{'Aspect': '소음', 'SentimentText': '소음이 섞여서 나는..."
...,...,...,...,...,...,...,...,...,...,...,...,...
96,114667,이전부터 구매하고 싶어서 계속 눈여겨보다 구매한 락클래식이에요. 포장을 제거하고 제...,SNS,가전,영상/음향가전,엠지텍 락클래식Q9900 (정품),1,290,61,20221120,-1.0,"[{'Aspect': '품질', 'SentimentText': '마감은 좀 문제가 ..."
97,114668,요즘 집에서 작업하면서 핸드폰으로 음악을 들으니 전화를 하거나 핸드폰을 이용할때 자...,SNS,가전,영상/음향가전,오아 아이브릭 휴대용 블루투스 미니 스피커,1,338,78,20221110,-1.0,"[{'Aspect': '디자인', 'SentimentText': '디자인이 좀 그렇..."
98,114669,"처음 들어보는 생소한 브랜드의 tv라 걱정하면서 구입했는데, 역시나 후회 중입니다....",SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,333,79,20221116,-1.0,"[{'Aspect': '소음', 'SentimentText': '별로 소리를 키우지..."
99,114670,최신 기종이라고 해서 기대했는데 구기종보다 못하네요. 소재가 별로여서 예쁜 디자인이...,SNS,가전,영상/음향가전,유맥스 139cm 무결점 UHD ／ UHD55L [스탠드형 자가 설치],1,324,74,20221116,-1.0,"[{'Aspect': '제조일/제조사', 'SentimentText': '최신 기종..."


In [170]:
# 모든 데이터프레임을 로드하여 하나의 데이터프레임으로 생성 
# 누적으로 데이터프레임이 결합되는 공간 -> 빈 데이터프레임
df = pd.DataFrame()
# file_list2 만큼 반복 실행하는 반복문을 생성
for file_name in file_list2:
    data = pd.read_json(file_name)
    df = pd.concat([df, data], axis= 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [171]:
pd.concat(
    [ pd.read_json(file_name) for file_name in file_list2 ]
).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [172]:
# python 기본 내장된 map() 함수 이용 -> map( funtion, list )
pd.concat(
    list(map( lambda file_name : pd.read_json(file_name) , file_list2))
).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [174]:
# 결측치의 개수 확인 
df['GeneralPolarity'].isna().sum()

np.int64(378)

In [175]:
# 결측치인 데이터들을 다른 변수에 저장 
na_df = df.loc[
    df['GeneralPolarity'].isna(), 
]
na_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378 entries, 13 to 88
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            378 non-null    int64  
 1   RawText          378 non-null    object 
 2   Source           378 non-null    object 
 3   Domain           378 non-null    object 
 4   MainCategory     378 non-null    object 
 5   ProductName      378 non-null    object 
 6   ReviewScore      378 non-null    int64  
 7   Syllable         378 non-null    int64  
 8   Word             378 non-null    int64  
 9   RDate            378 non-null    int64  
 10  GeneralPolarity  0 non-null      float64
 11  Aspects          378 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 38.4+ KB


In [176]:
# 결측치를 제거 df에서 
df.dropna(inplace=True)

In [177]:
# 특정 컬럼 두개를 제외한 다른 컬럼은 제외 -> 특정 컬럼 2개만 선택 
df = df[['RawText', 'GeneralPolarity']]

In [178]:
# 특정 컬럼의 이름을 변경
# reaname()
df.rename(columns={
    'GeneralPolarity' : 'labels'
}, inplace=True)

In [179]:
df['labels'].value_counts()

labels
 1.0    2220
 0.0     944
-1.0     514
Name: count, dtype: int64

In [180]:
# RawText는 문자의 정규화
# labels는 -1인 데이터는 0으로 타입을 int로 변경

# labels컬럼의 데이터의 타입을 int로 고정 시킨다. 
df['labels'] = df['labels'].astype(int)

In [181]:
import re

In [182]:
def normalize(x):
    # x -> apply()함수를 이용하여 들어오는 Series 형태의 데이터
    x['RawText'] = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", x['RawText'])
    x['RawText'] = re.sub(r"\s+", " ", x['RawText'])
    if x['labels'] == -1:
        x['labels'] = 0
    return x

df = df.apply(normalize, axis=1) 
# 컬럼을 기준으로 데이터를 나눠서 보여준다.
# axis = 0 
# df['RawText']를 x에 대입하여 한번 실행
# df['labels']를 x에 대입하여 한번 실행
# axis = 1
# df.iloc[0, ] x대입
# df.ilox[1, ] x대입 ....

In [183]:
df['labels'].value_counts()

labels
1    2220
0    1458
Name: count, dtype: int64

In [184]:
# train, test 데이터셋으로 분할 -> labels의 데이터로 계층화
train_df, test_df = train_test_split(
    df, test_size=0.1, random_state=42, stratify=df['labels']
)
len(train_df)

3310

In [185]:
# DataFrame -> Dataset 형태로 파싱 
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

In [186]:
# Tokenizer 
MODEL_NAME = 'skt/kobert-base-v1'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast = False)

In [187]:
def tok_fn(batch):
    # batch -> Dataset에서 map() 함수의 기본값이 데이터를 특정 개수로 묶어서 보낸다. 
    result = tokenizer(batch['RawText'], truncation=True, max_length=128)
    # 토큰의 개수의 제한을 두고 해당 토큰보다 길다면 잘라준다. 
    return result

train_tok = train_ds.map(tok_fn, batched=True, remove_columns=['RawText'])
test_tok = test_ds.map(tok_fn, batched=True, remove_columns=['RawText'])

Map: 100%|██████████| 368/368 [00:00<00:00, 2807.68 examples/s]


In [188]:
train_tok

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3310
})

In [195]:
# BertModel + Linbear 모델의 정의 
class BERTClsHead(nn.Module):
    def __init__(self, model_name, num_label = 2, dropout = 0.1):
        super().__init__()
        # 기존의 학습이 된 모델 로드
        self.backbone = BertModel.from_pretrained(model_name)
        # 로드한 모델에서 출력 차원의 개수를 저장 
        hidden = self.backbone.config.hidden_size

        # 일정 부분 소실시키는 Dropout
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden, num_label)

        # 작업의 안정성을 위해서 tokenizer에서 사용하는 패딩 토큰의 id 를 모델에 패딩 id 대입 
        self.backbone.config.pad_token_id = tokenizer.pad_token_id

    # 순전파 함수 
    def forward(self, input_ids = None, attention_mask = None, 
                labels = None, **kwargs):
        out = self.backbone(input_ids = input_ids, attention_mask = attention_mask)

        pooled = out.last_hidden_state[:, 0]

        logits = self.classifier(
            self.dropout(pooled)
        )

        result = {"Logits" : logits}

        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            result['loss'] = loss
        return result


In [196]:
# 모델을 생성 
model = BERTClsHead(MODEL_NAME, num_label=2)

In [197]:
# 평가에서 사용할 함수 정의 
def metrics(eval_pred):
    logits, y = eval_pred
    pred = logits.argmax(-1)
    return {
        'accuracy_score' : accuracy_score(y, pred), 
        'f1_score' : f1_score(y, pred)
    }

In [198]:
# Trainer가 사용할 설정 계수를 정의 
args = TrainingArguments(
    output_dir= './kobert_from_bertmodel', 
    eval_strategy= 'epoch', 
    save_strategy='epoch', 
    num_train_epochs=2,             # 실제 횟수 -> 일반적으로 3-5회 
    learning_rate=5e-5,             # 5e-5, 4e-5, 3e-5
    weight_decay=0.01, 
    warmup_ratio=0.01, 
    logging_steps=50, 
    load_best_model_at_end=True,
    metric_for_best_model='f1', 
    greater_is_better=True, 
    report_to= []
)

In [199]:
trainer = Trainer(
    model = model, 
    args = args, 
    train_dataset=train_tok, 
    eval_dataset=test_tok, 
    tokenizer = tokenizer, 
    compute_metrics= metrics
)

C:\Users\ekfla\AppData\Local\Temp\ipykernel_4588\2748943638.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [200]:
eval_res = trainer.evaluate()

print(eval_res)

{'eval_loss': 0.6936672329902649, 'eval_model_preparation_time': 0.0032, 'eval_accuracy_score': 0.5108695652173914, 'eval_f1_score': 0.6356275303643725, 'eval_runtime': 23.7831, 'eval_samples_per_second': 15.473, 'eval_steps_per_second': 1.934}


In [204]:
# 새로운 문장에 대해서 평가 
samples = na_df['RawText'].head().to_list()

In [205]:
enc = tokenizer(
    samples, 
    return_tensors = 'pt', 
    padding = True, 
    truncation = True, 
    max_length = 128
)

In [206]:
with torch.no_grad():
    out = model(**enc)
    probs = torch.softmax(out['Logits'], dim=-1)
    

In [209]:
for s, p in zip(samples, probs):
    print(f"""
    리뷰 데이터 : {s[:40]}
    부정 : {p[0]:.3f} 긍정 : {p[1]:.3f} 예측 : {p.argmax()}
""")


    리뷰 데이터 : 귀에서 자꾸 빠져요.귀에 꼽는재 질이 미끄러운 재질이라 작은 소품이지만 
    부정 : 0.446 긍정 : 0.554 예측 : 1


    리뷰 데이터 : 아이를 출산한 기념으로 TV를 바꿨습니다. 그전에 쓰던 TV가 꽤나 무거
    부정 : 0.490 긍정 : 0.510 예측 : 1


    리뷰 데이터 : 화면에 노이즈가 생깁니다. 저희 가족 중에 아무도 TV 화면을 건드리거나
    부정 : 0.448 긍정 : 0.552 예측 : 1


    리뷰 데이터 : 이번에 이사하면서 우리 따님께서 방에 TV가 있으면 좋겠다고 하여, 방에
    부정 : 0.486 긍정 : 0.514 예측 : 1


    리뷰 데이터 : 기존에 사용하던 무선이어폰이 오래되어서 배터리가 광탈하는 바람에 새로운 
    부정 : 0.454 긍정 : 0.546 예측 : 1

